# AI DashCam Accident Video Summary

AI-powered system condenses DashCam footage into brief videos for streamlined accident insurance claims processing.

This sample notebook shows you how to execute AI DashCam Accident Video Summary Algorithm from AWS Marketplace

 **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

### Pre-requisites:

1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
2. Ensure that IAM role used has **AmazonSageMakerFullAccess**
3. Some hands-on experience using Amazon SageMaker.
4. To use this algorithm successfully, ensure that:  
    A. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the            AWS account used:  
&nbsp;&nbsp;&nbsp;&nbsp;            a. **aws-marketplace:ViewSubscriptions**  
&nbsp;&nbsp;&nbsp;&nbsp;            b. **aws-marketplace:Unsubscribe**  
&nbsp;&nbsp;&nbsp;&nbsp;            c. **aws-marketplace:Subscribe**  
    B. or your AWS account has a subscription to For Seller to update:  AI DashCam Accident Video Summary.


### Contents

1. [Subscribe to the algorithm]()
2. [Prepare dataset]()  
    A. [Dataset format expected by the algorithm]()  
    B. [Configure dataset]()  
    C. [Upload datasets to Amazon S3]()  
3. [Execute optimization model]()  
    A. [Set up environment]()    
    B. [Execute model]()  
    C. [Visualize Output]()  
4. [Clean-up]()  
    A. [Unsubscribe to the listing (optional))]()

### 1. Subscribe to the algorithm

To subscribe to the algorithm:  
&nbsp; 1. Open the algorithm listing page **AI DashCam Accident Video Summary**  
&nbsp; 2. On the AWS Marketplace listing, click on **Continue to subscribe** button.  
&nbsp; 3. On the **Subscribe to this software** page, review and click on "**Accept Offer**" if you agree with EULA, pricing, and support terms.  
&nbsp; 4. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
algo_arn = "your-algo-arn"

### 2. Prepare dataset

In [ ]:
import os
import json 
import uuid
import boto3
import pickle
import base64
import tarfile
from pprint import pprint
from PIL import Image

import numpy as np
import pandas as pd
from zipfile import ZipFile

import urllib.request
from urllib.parse import urlparse

import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role



#### A. Dataset format expected by the algorithm

The algorithm requires data in the format as described for best results:  

&nbsp; 1. Input File name should be input_zip.zip.  
&nbsp; 2. The input data files must contain one csv file and one directory containing videos.  
&nbsp; 3. For detailed instructions, please refer sample notebook and algorithm input details.  

#### B. Configure dataset

In [3]:
training_dataset="Input/training/input_zip.zip"

#### C. Upload datasets to Amazon S3

In [4]:
role = get_execution_role()

In [ ]:
sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
bucket

In [ ]:
# training input location
common_prefix = "video-summarization"
training_input_prefix = common_prefix + "/training-input-data"
TRAINING_WORKDIR = "Input/training" #Input directory in Jupyter Server
training_input = sagemaker_session.upload_data(TRAINING_WORKDIR, key_prefix=training_input_prefix) #uploads data from jupyter server to S3
print("Training input uploaded to " + training_input)

### 3. Execute the training process

Now that dataset is available in an accessible Amazon S3 bucket, we are ready to execute a training pipeline to get clean sentiment class labels using clean-sentiment-classification-labels Algorithm.

#### 3.1 Set up environment

In [7]:
output_location = 's3://{}/video-summarization/{}'.format(bucket, 'Output')

#### 3.2 Execute model

For information on creating an Estimator object, see [documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

In [8]:
# training_instance_type="ml.m5.4xlarge"

In [9]:
training_instance_type="ml.g4dn.2xlarge"

In [10]:
#Create an estimator object for running a training job
estimator = sage.algorithm.AlgorithmEstimator(
    algorithm_arn=algo_arn,
    base_job_name="video-summarization",
    role=role,
    train_instance_count=1,
    train_instance_type=training_instance_type,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
    instance_count=1,
    instance_type=training_instance_type
)

#Run the training job.
estimator.fit({"training": training_input})

INFO:sagemaker:Creating training-job with name: video-summarization-2024-08-14-05-16-07-639


2024-08-14 05:16:09 Starting - Starting the training job...
2024-08-14 05:16:23 Starting - Preparing the instances for training...
2024-08-14 05:17:06 Downloading - Downloading the training image.....................
2024-08-14 05:20:28 Training - Training image download completed. Training in progress...==========
== CUDA ==
CUDA Version 12.1.1
Container image Copyright (c) 2016-2023, NVIDIA CORPORATION & AFFILIATES. All rights reserved.
This container image and its contents are governed by the NVIDIA Deep Learning Container License.
By pulling and using the container, you accept the terms and conditions of this license:
https://developer.nvidia.com/ngc/nvidia-deep-learning-container-license
A copy of this license is made available in this container at /NGC-DL-CONTAINER-LICENSE for your convenience.
2024-08-14 05:20:53.136656: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors fro

#015Batch:   8%|██▉                                 | 8/100 [00:03<00:31,  2.91it/s]#033[A
SCORES torch.Size([50, 1, 1])
SCORES torch.Size([50, 1, 1]) torch.Size([50, 1, 500]) torch.Size([50, 1, 1])
#015Batch:   9%|███▏                                | 9/100 [00:03<00:30,  2.94it/s]#033[A#015                                                                                #015
original_p: 0.507, fake_p: 0.477, uniform_p: 0.477
#015                                                                                #015#033[A#015Epoch:   0%|                                             | 0/10 [00:03<?, ?it/s]
#015Batch:   9%|███▏                                | 9/100 [00:03<00:30,  2.94it/s]#033[A#015                                                                                #015
recon loss 0.329, prior loss: 28.912, sparsity loss: 0.020
#015                                                                                #015#033[A#015Epoch:   0%|                                            

#015Batch:  38%|█████████████▎                     | 38/100 [00:13<00:20,  3.00it/s]#033[A
SCORES torch.Size([50, 1, 1])
SCORES torch.Size([50, 1, 1]) torch.Size([50, 1, 500]) torch.Size([50, 1, 1])
#015Batch:  39%|█████████████▋                     | 39/100 [00:13<00:20,  3.00it/s]#033[A#015                                                                                #015
original_p: 0.985, fake_p: 0.025, uniform_p: 0.030
#015                                                                                #015#033[A#015Epoch:   0%|                                             | 0/10 [00:13<?, ?it/s]
#015Batch:  39%|█████████████▋                     | 39/100 [00:13<00:20,  3.00it/s]#033[A#015                                                                                #015
recon loss 14.363, prior loss: 1.557, sparsity loss: 0.574
#015                                                                                #015#033[A#015Epoch:   0%|                                            

#015Batch:  68%|███████████████████████▊           | 68/100 [00:23<00:10,  2.99it/s]#033[A
SCORES torch.Size([50, 1, 1])
SCORES torch.Size([50, 1, 1]) torch.Size([50, 1, 500]) torch.Size([50, 1, 1])
#015Batch:  69%|████████████████████████▏          | 69/100 [00:23<00:10,  2.99it/s]#033[A#015                                                                                #015
original_p: 0.936, fake_p: 0.001, uniform_p: 0.001
#015                                                                                #015#033[A#015Epoch:   0%|                                             | 0/10 [00:23<?, ?it/s]
#015Batch:  69%|████████████████████████▏          | 69/100 [00:23<00:10,  2.99it/s]#033[A#015                                                                                #015
recon loss 18.115, prior loss: 0.229, sparsity loss: 0.260
#015                                                                                #015#033[A#015Epoch:   0%|                                            

...

#015Batch:   0%|                                            | 0/100 [00:00<?, ?it/s]#033[A#015                                                                                #015
original_p: 0.952, fake_p: 0.942, uniform_p: 0.942
#015                                                                                #015#033[A#015Epoch:  90%|█████████████████████████████████▎   | 9/10 [04:58<00:32, 32.97s/it]
#015Batch:   0%|                                            | 0/100 [00:00<?, ?it/s]#033[A#015                                                                                #015
recon loss 1.879, gan loss: -5.752
#015                                                                                #015#033[A#015Epoch:  90%|█████████████████████████████████▎   | 9/10 [04:58<00:32, 32.97s/it]
#015Batch:   0%|                                            | 0/100 [00:00<?, ?it/s]#033[A
SCORES torch.Size([50, 1, 1])
SCORES torch.Size([50, 1, 1]) torch.Size([50, 1, 500]) torch.Size([50, 1, 1])

#015Batch:  31%|██████████▊                        | 31/100 [00:09<00:22,  3.07it/s]#033[A
SCORES torch.Size([50, 1, 1])
SCORES torch.Size([50, 1, 1]) torch.Size([50, 1, 500]) torch.Size([50, 1, 1])
#015Batch:  32%|███████████▏                       | 32/100 [00:10<00:22,  3.07it/s]#033[A#015                                                                                #015
original_p: 0.675, fake_p: 0.032, uniform_p: 0.033
#015                                                                                #015#033[A#015Epoch:  90%|█████████████████████████████████▎   | 9/10 [05:08<00:32, 32.97s/it]
#015Batch:  32%|███████████▏                       | 32/100 [00:10<00:22,  3.07it/s]#033[A#015                                                                                #015
recon loss 9.989, prior loss: 0.015, sparsity loss: 0.031
#015                                                                                #015#033[A#015Epoch:  90%|█████████████████████████████████▎   | 9/10 [

SCORES torch.Size([50, 1, 1])
SCORES torch.Size([50, 1, 1]) torch.Size([50, 1, 500]) torch.Size([50, 1, 1])
#015Batch:  47%|████████████████▍                  | 47/100 [00:15<00:17,  3.06it/s]#033[A#015                                                                                #015
original_p: 0.883, fake_p: 0.015, uniform_p: 0.015
#015                                                                                #015#033[A#015Epoch:  90%|█████████████████████████████████▎   | 9/10 [05:13<00:32, 32.97s/it]
#015Batch:  47%|████████████████▍                  | 47/100 [00:15<00:17,  3.06it/s]#033[A#015                                                                                #015
recon loss 13.891, gan loss: -0.155
#015                                                                                #015#033[A#015Epoch:  90%|█████████████████████████████████▎   | 9/10 [05:13<00:32, 32.97s/it]
SCORES torch.Size([50, 1, 1])
SCORES torch.Size([50, 1, 1]) torch.Size([50, 1, 500]) torc

#015Batch:  94%|████████████████████████████████▉  | 94/100 [00:30<00:01,  3.10it/s]#033[A#015                                                                                #015
original_p: 0.058, fake_p: 0.020, uniform_p: 0.020
#015                                                                                #015#033[A#015Epoch:  90%|█████████████████████████████████▎   | 9/10 [05:28<00:32, 32.97s/it]
#015Batch:  94%|████████████████████████████████▉  | 94/100 [00:30<00:01,  3.10it/s]#033[A#015                                                                                #015
recon loss 3.278, prior loss: 0.057, sparsity loss: 0.039
#015                                                                                #015#033[A#015Epoch:  90%|█████████████████████████████████▎   | 9/10 [05:28<00:32, 32.97s/it]
SCORES torch.Size([50, 1, 1])
SCORES torch.Size([50, 1, 1]) torch.Size([50, 1, 500]) torch.Size([50, 1, 1])
#015Batch:  94%|████████████████████████████████▉  | 94/100 [00:30<


2024-08-14 05:28:18 Uploading - Uploading generated training model
2024-08-14 05:28:18 Completed - Training job completed
Training seconds: 686
Billable seconds: 686


See this [blog-post](https://aws.amazon.com/blogs/machine-learning/easily-monitor-and-visualize-metrics-while-training-models-on-amazon-sagemaker/) for more information how to visualize metrics during the process. You can also open the training job from [Amazon SageMaker console](https://us-east-2.console.aws.amazon.com/sagemaker/home?region=us-east-2#/jobs/) and monitor the metrics/logs in **Monitor** section.

In [ ]:
#output is available on following path
estimator.output_path

#### 3.3 Inspect the Output in S3

In [12]:
parsed_url = urlparse(estimator.output_path)
bucket_name = parsed_url.netloc
file_key = parsed_url.path[1:]+'/'+estimator.latest_training_job.job_name+'/output/'+"model.tar.gz"

s3_client = sagemaker_session.boto_session.client('s3')
response = s3_client.get_object(Bucket = sagemaker_session.default_bucket(), Key = file_key) 

In [13]:
bucketFolder = estimator.output_path.rsplit('/')[3] +'/Output/'+ estimator.latest_training_job.job_name+'/output/'+"model.tar.gz"

### 4. Deploy model and verify results

Now you can deploy the model for performing real-time inference.

In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

In [15]:
model_name='video-summarization'

content_type='application/zip'

real_time_inference_instance_type='ml.p3.8xlarge'
batch_transform_inference_instance_type='ml.p3.8xlarge'

#### 4.1 Deploy trained model

In [16]:
from sagemaker.serializers import CSVSerializer

predictor = estimator.deploy(1, real_time_inference_instance_type)

INFO:sagemaker:Creating model package with name: video-summarization-2024-08-14-05-29-00-334


.........

INFO:sagemaker:Creating model with name: video-summarization-2024-08-14-05-29-00-334


INFO:sagemaker:Creating endpoint-config with name video-summarization-2024-08-14-05-29-00-334
INFO:sagemaker:Creating endpoint with name video-summarization-2024-08-14-05-29-00-334


-------------!

#### 4.2 Create input payload

In [17]:
file_name = 'Input/inference/Inference_Videos.zip'

#### 4.3 Perform real-time inference

In [18]:
output_file_name = 'output/inf_output.zip'

In [19]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $predictor.endpoint_name \
    --body fileb://$file_name \
    --content-type $content_type \
    --region $sagemaker_session.boto_region_name \
    $output_file_name

{
    "ContentType": "application/zip",
    "InvokedProductionVariant": "AllTraffic"
}


#### 4.4 Check output

In [20]:
output_zip_path='output/inf_output.zip'
output='output/'
with ZipFile (output_zip_path, "r") as data_zip:
    print('Extracting all the files now...')
    data_zip.extractall(output)
print('Please check output folder for results')

Extracting all the files now...
Please check output folder for results


### 5. Clean-up

In [21]:
#predictor=sage.predictor.Predictor(model_name, sagemaker_session,content_type)
predictor.delete_endpoint(delete_endpoint_config=True)

INFO:sagemaker:Deleting endpoint configuration with name: video-summarization-2024-08-14-05-29-00-334
INFO:sagemaker:Deleting endpoint with name: video-summarization-2024-08-14-05-29-00-334


#### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://us-east-2.console.aws.amazon.com/sagemaker/home?region=us-east-2#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model.

#### Steps to unsubscribe to product from AWS Marketplace:

&nbsp; 1. Navigate to **Machine Learning tab** on **Your Software subscriptions page**.  
&nbsp; 2. Locate the listing that you want to cancel the subscription for, and then choose **Cancel Subscription** to cancel the subscription.